In [1]:
%pip install rdflib
%pip install owlrl
%pip install pandas
from rdflib import Graph, Namespace
import pandas as pd

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from rdflib import Graph, Namespace
from rdflib.namespace import RDF, XSD

# Load your original SPHN data graph
g = Graph()
g.parse("sphn_instance_administrativecase.ttl", format="ttl")  # replace with your actual SPHN TTL file

# Define namespaces
sphn = Namespace("https://biomedit.ch/rdf/sphn-ontology/sphn#")
sulo = Namespace("https://w3id.org/sulo/")
ex = Namespace("https://example.org/resource/")
print(f"{len(g)} Triples")

11 Triples


In [3]:
# Define the transformation SPARQL query
transformation_query = """
PREFIX sphn: <https://biomedit.ch/rdf/sphn-ontology/sphn#>
PREFIX sulo: <https://w3id.org/sulo/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX : <https://example.org/resource/>

CONSTRUCT {
  ?case a :AdministrativeCase .

  ?case sulo:hasFeature [ a sulo:InformationObject ; sulo:hasValue ?identifier ] .

  ?case sulo:occursIn [ a sulo:StartTime ; sulo:hasValue ?startTime ] .

  ?case sulo:isReferredIn ?dataFile .

  ?case sulo:isLocatedIn ?location .

  ?case sulo:hasPart [
    a :AdmissionProcess ;
    sulo:occursIn [ a sulo:Time ; sulo:hasValue ?admissionTime ] ;
    sulo:isLocatedIn ?originLocation
  ] .

  ?case sulo:hasPart [
    a :DischargeProcess ;
    sulo:occursIn [ a sulo:Time ; sulo:hasValue ?dischargeTime ] ;
    sulo:isLocatedIn ?dischargeLocation
  ] .

  ?case sulo:hasParticipant [
    a :PatientRole ;
    sulo:isFeatureOf ?subject
  ] .

  ?case sulo:hasPart ?careHandling .
}
WHERE {
  ?case a sphn:AdministrativeCase .

  OPTIONAL { ?case sphn:hasIdentifier ?identifier. }
  OPTIONAL { ?case sphn:hasDateTime ?startTime. }
  OPTIONAL { ?case sphn:hasDataFile ?dataFile. }
  OPTIONAL { ?case sphn:hasLocation ?location. }
  OPTIONAL { ?case sphn:hasAdmissionDateTime ?admissionTime. }
  OPTIONAL { ?case sphn:hasOriginLocation ?originLocation. }
  OPTIONAL { ?case sphn:hasDischargeDateTime ?dischargeTime. }
  OPTIONAL { ?case sphn:hasDischargeLocation ?dischargeLocation. }
  OPTIONAL { ?case sphn:hasSubjectPseudoIdentifier ?subject. }
  OPTIONAL { ?case sphn:hasCareHandling ?careHandling. }
}
"""

# Run the transformation query
transformed_graph = g.query(transformation_query).graph

# Bind prefixes for cleaner output
transformed_graph.bind("sulo", sulo)
transformed_graph.bind("sphn", sphn)
transformed_graph.bind("", ex)

# Save the transformed SULO instance to Turtle format
transformed_graph.serialize("transformed_sulo_instance.ttl", format="turtle")

print("Transformation complete. Output written to 'transformed_sulo_instance_administrativecase.ttl'")


Transformation complete. Output written to 'transformed_sulo_instance_administrativecase.ttl'


In [8]:
query = """
PREFIX sphn: <https://biomedit.ch/rdf/sphn-ontology/sphn#>
PREFIX sulo: <https://w3id.org/sulo/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX : <https://example.org/resource/>

CONSTRUCT {
  ?case a :AdministrativeCase ;

        # Identifier
        sulo:hasFeature [ a sulo:InformationObject ; sulo:hasValue ?caseIdentifier ] ;

        # Shared identifier
        sulo:hasFeature [ a sulo:InformationObject, :SharedIdentifier ; sulo:hasValue ?sharedId ] ;

        # Subject (patient)
        sulo:hasParticipant [
          a :PatientRole ;
          sulo:isFeatureOf [
            a :Patient ;
            sulo:hasFeature [ a sulo:InformationObject ; sulo:hasValue ?subjectId ]
          ]
        ] ;

        # Data Provider
        sulo:hasParticipant [
          a :DataProviderRole ;
          sulo:isFeatureOf [
            a :DataProvider ;
            sulo:hasFeature [ a sulo:InformationObject ; sulo:hasValue ?dataProviderId ]
          ]
        ] ;

        # Source System
        sulo:hasParticipant [
          a :SourceSystemRole ;
          sulo:isFeatureOf :sourceSystemA
        ] ;

        # Care Handling via PRO pattern
        sulo:hasParticipant [
          a :CareHandlingRole ;
          sulo:isFeatureOf [
            a :CareProviderInstitute ;
            sulo:hasFeature [
              a sulo:InformationObject ;
              sulo:hasValue ?careHandling
            ]
          ]
        ] ;

        # Admission & Discharge
        sulo:hasPart ?admission ;
        sulo:hasPart ?discharge .

  # Admission process
  ?admission a :AdmissionProcess ;
             sulo:hasPart :preAdmissionX ;
             sulo:isLocatedIn ?originLocation ;
             sulo:atTime [ a sulo:Time ; sulo:hasValue ?admissionTime ] .

  # PreAdmission process
  :preAdmissionX a :PreAdmissionProcess ;
                 sulo:isLocatedIn ?originLocation .

  # Discharge process
  ?discharge a :DischargeProcess ;
             sulo:isLocatedIn ?targetLocation ;
             sulo:atTime [ a sulo:Time ; sulo:hasValue ?dischargeTime ] .

  # Source System details
  :sourceSystemA a :SourceSystem ;
                 sulo:refersTo :primarySystem1 ;
                 sulo:hasFeature [
                   a :SourceSystemCategory ;
                   sulo:hasValue ?category
                 ],
                 [
                   a :Purpose ;
                   sulo:hasValue ?purpose
                 ],
                 [
                   a :Name ;
                   sulo:hasValue ?systemName
                 ] .
}
WHERE {
  ?case a sphn:AdministrativeCase .

  OPTIONAL { ?case sphn:hasIdentifier ?caseIdentifier. }

  OPTIONAL { ?case sphn:hasCareHandling ?careHandling. }

  OPTIONAL {
    ?case sphn:hasSubjectPseudoIndentifier ?subject .
    ?subject sphn:hasIdentifier ?subjectId .
    OPTIONAL { ?subject sphn:hasSharedIdentifier ?sharedId. }
    OPTIONAL {
      ?subject sphn:hasDataProvider ?dataProvider .
      ?dataProvider :hasIdentifier ?dataProviderId .
    }
  }

  OPTIONAL {
    ?case sphn:hasSourceSystem ?system .
    OPTIONAL { ?system :hasName ?systemName. }
    OPTIONAL { ?system :hasPrimarySystem ?primarySystem1. }
    OPTIONAL { ?system :hasPurpose ?purpose. }
    OPTIONAL { ?system :hasCategory ?category. }
  }

  OPTIONAL {
    ?case sphn:hasAdmission ?admission .
    ?admission sphn:hasOriginLocation ?originLocation .
    ?admission sphn:hasDateTime ?admissionTime .
  }

  OPTIONAL {
    ?case sphn:hasDischarge ?discharge .
    ?discharge sphn:hasTargetLocation ?targetLocation .
    ?discharge sphn:hasDateTime ?dischargeTime .
  }
}


"""

In [9]:
from rdflib import Graph, Namespace

# Load graph
g = Graph()
g.parse("sphn_instance_administrativecase_v2025.ttl", format="ttl")

# Namespaces
sphn = Namespace("https://biomedit.ch/rdf/sphn-ontology/sphn#")
sulo = Namespace("https://w3id.org/sulo/")
ex = Namespace("https://example.org/resource/")

# Execute query
transformed_graph = g.query(query).graph
transformed_graph.bind("sulo", sulo)
transformed_graph.bind("", ex)

# Save output
transformed_graph.serialize("transformed_sulo_instance_v2025_test1.ttl", format="turtle")
print("Transformation complete. Output written to 'sulo_output_2025.ttl'")


Transformation complete. Output written to 'sulo_output_2025.ttl'


In [23]:
sulotosphn_query = """

PREFIX sphn: <https://biomedit.ch/rdf/sphn-ontology/sphn#>
PREFIX sulo: <https://w3id.org/sulo/>
PREFIX : <https://example.org/resource/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

CONSTRUCT {
  ?case a sphn:AdministrativeCase ;
        sphn:hasIdentifier ?id ;
        sphn:hasSharedIdentifier ?sharedId ;
        sphn:hasCareHandling ?careValue ;
        sphn:hasSubjectPseudoIdentifier ?subject ;
        sphn:hasSourceSystem ?sourceSystem ;
        sphn:hasAdmission ?admission ;
        sphn:hasDischarge ?discharge .

  ?subject a sphn:SubjectPseudoIdentifier ;
           sphn:hasIdentifier ?subjectId ;
           sphn:hasDataProvider ?dataProvider .

  ?dataProvider a sphn:SourceSystem ;
                sphn:hasIdentifier ?dataProviderId .

  ?sourceSystem a sphn:SourceSystem ;
                sphn:hasName ?name ;
                sphn:hasPrimarySystem ?primarySystem ;
                sphn:hasCategory ?category ;
                sphn:hasPurpose ?purpose .

  ?admission a sphn:Admission ;
             sphn:hasOriginLocation ?originLocation ;
             sphn:hasDateTime ?admissionTime .

  ?discharge a sphn:Discharge ;
             sphn:hasTargetLocation ?dischargeLocation ;
             sphn:hasDateTime ?dischargeTime .
}
WHERE {
  ?case a :AdministrativeCase .

  # Identifier and shared ID
  ?case sulo:hasFeature ?idNode .
  ?idNode sulo:hasValue ?id .
  FILTER(isLiteral(?id))

  OPTIONAL {
    ?case sulo:hasFeature ?sharedNode .
    ?sharedNode a :SharedIdentifier ;
                sulo:hasValue ?sharedId .
  }

  # CareHandling (PRO pattern)
  OPTIONAL {
    ?case sulo:hasParticipant ?careRole .
    ?careRole a :CareHandlingRole ;
              sulo:isFeatureOf ?careProvider .
    ?careProvider sulo:hasFeature ?careFeat .
    ?careFeat sulo:hasValue ?careValue .
  }

  # Subject and DataProvider
  OPTIONAL {
    ?case sulo:hasParticipant ?patRole .
    ?patRole a :PatientRole ;
             sulo:isFeatureOf ?subject .
    ?subject sulo:hasFeature ?subjectFeat .
    ?subjectFeat sulo:hasValue ?subjectId .
  }

  OPTIONAL {
    ?case sulo:hasParticipant ?dpRole .
    ?dpRole a :DataProviderRole ;
           sulo:isFeatureOf ?dataProvider .
    ?dataProvider sulo:hasFeature ?dpFeat .
    ?dpFeat sulo:hasValue ?dataProviderId .
  }

  # Source System
  OPTIONAL {
    ?case sulo:hasParticipant ?ssRole .
    ?ssRole a :SourceSystemRole ;
            sulo:isFeatureOf ?sourceSystem .

    ?sourceSystem sulo:hasFeature ?nameNode .
    ?nameNode a :Name ;
              sulo:hasValue ?name .

    ?sourceSystem sulo:hasFeature ?catNode .
    ?catNode a :SourceSystemCategory ;
             sulo:hasValue ?category .

    ?sourceSystem sulo:hasFeature ?purpNode .
    ?purpNode a :Purpose ;
              sulo:hasValue ?purpose .

    ?sourceSystem sulo:refersTo ?primarySystem .
  }

  # Admission
  OPTIONAL {
    ?case sulo:hasPart ?admission .
    ?admission a :AdmissionProcess ;
               sulo:isLocatedIn ?originLocation ;
               sulo:atTime ?adTimeNode .
    ?adTimeNode sulo:hasValue ?admissionTime .
  }

  # Discharge
  OPTIONAL {
    ?case sulo:hasPart ?discharge .
    ?discharge a :DischargeProcess ;
               sulo:isLocatedIn ?dischargeLocation ;
               sulo:atTime ?disTimeNode .
    ?disTimeNode sulo:hasValue ?dischargeTime .
  }
}


"""

In [24]:
# Namespaces
sulo = Namespace("https://w3id.org/sulo/")
sphn = Namespace("https://biomedit.ch/rdf/sphn-ontology/sphn#")
ex = Namespace("https://example.org/resource/")

# Load SULO data
g = Graph()
g.parse("transformed_sulo_instance_v2025_test1.ttl", format="turtle")


# Run the transformation
transformed_graph = g.query(sulotosphn_query).graph

# Bind namespaces
transformed_graph.bind("sulo", sulo)
transformed_graph.bind("sphn", sphn)
transformed_graph.bind("", ex)

# Preview
print(transformed_graph.serialize(format="turtle"))

# Save result
transformed_graph.serialize("sulotosphn-output.ttl", format="turtle")


@prefix : <https://example.org/resource/> .
@prefix sphn: <https://biomedit.ch/rdf/sphn-ontology/sphn#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:case2025 a sphn:AdministrativeCase ;
    sphn:hasAdmission :admissionX ;
    sphn:hasCareHandling :careHandling2025 ;
    sphn:hasDischarge :dischargeX ;
    sphn:hasIdentifier "ADM-2025-001",
        "shared-abc-999" ;
    sphn:hasSharedIdentifier "shared-abc-999" ;
    sphn:hasSourceSystem :sourceSystemA ;
    sphn:hasSubjectPseudoIdentifier [ a sphn:SubjectPseudoIdentifier ;
            sphn:hasDataProvider [ a sphn:SourceSystem ;
                    sphn:hasIdentifier "DP-001-HOSP" ] ;
            sphn:hasIdentifier "SUBJ-789XYZ" ] .

:admissionX a sphn:Admission ;
    sphn:hasDateTime "2025-06-10T08:15:00+00:00"^^xsd:dateTime ;
    sphn:hasOriginLocation :ER_Department .

:dischargeX a sphn:Discharge ;
    sphn:hasDateTime "2025-06-15T12:00:00+00:00"^^xsd:dateTime ;
    sphn:hasTargetLocation :RecoveryUnit .

:sourceSystemA 

<Graph identifier=N69480301ec754dcbae1ff5921cee2d52 (<class 'rdflib.graph.Graph'>)>

In [26]:
from rdflib import Graph
from pyshacl import validate

# Load data graph (SPHN output)
data_graph = Graph()
data_graph.parse("sulotosphn-output.ttl", format="turtle")

# Load SHACL shape graph
shacl_graph = Graph()
shacl_graph.parse("sphn-shacl-shape.ttl", format="turtle")


<Graph identifier=N05ad43c2880b4b588a3ccf86923bbd12 (<class 'rdflib.graph.Graph'>)>

In [27]:
conforms, report_graph, report_text = validate(
    data_graph=data_graph,
    shacl_graph=shacl_graph,
    inference="rdfs",
    serialize_report_graph=True
)

print("Conforms:", conforms)
print(report_text)

# Optional: save SHACL validation report
with open("shacl-validation-report.ttl", "wb") as f:
    f.write(report_graph)


Conforms: True
Validation Report
Conforms: True



In [20]:
for s, p, o in g.triples((None, RDF.type, None)):
    print(s, "a", o)


https://example.org/resource/case2025 a https://example.org/resource/AdministrativeCase
nc4b96a77540b440d95e5afb85f4972ebb1 a https://example.org/resource/SharedIdentifier
nc4b96a77540b440d95e5afb85f4972ebb1 a https://w3id.org/sulo/InformationObject
nc4b96a77540b440d95e5afb85f4972ebb2 a https://w3id.org/sulo/InformationObject
nc4b96a77540b440d95e5afb85f4972ebb5 a https://w3id.org/sulo/InformationObject
nc4b96a77540b440d95e5afb85f4972ebb8 a https://w3id.org/sulo/InformationObject
nc4b96a77540b440d95e5afb85f4972ebb12 a https://w3id.org/sulo/InformationObject
nc4b96a77540b440d95e5afb85f4972ebb3 a https://example.org/resource/CareHandlingRole
nc4b96a77540b440d95e5afb85f4972ebb4 a https://example.org/resource/CareProviderInstitute
nc4b96a77540b440d95e5afb85f4972ebb6 a https://example.org/resource/PatientRole
nc4b96a77540b440d95e5afb85f4972ebb7 a https://example.org/resource/Patient
nc4b96a77540b440d95e5afb85f4972ebb9 a https://example.org/resource/SourceSystemRole
nc4b96a77540b440d95e5afb85

In [19]:
print(g.serialize(format="turtle")[:1000])  # preview the first 1000 characters


@prefix : <https://example.org/resource/> .
@prefix sulo: <https://w3id.org/sulo/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:case2025 a :AdministrativeCase ;
    sulo:hasFeature [ a :SharedIdentifier,
                sulo:InformationObject ;
            sulo:hasValue "shared-abc-999" ],
        [ a sulo:InformationObject ;
            sulo:hasValue "ADM-2025-001" ] ;
    sulo:hasPart :admissionX,
        :dischargeX ;
    sulo:hasParticipant [ a :DataProviderRole ;
            sulo:isFeatureOf [ a :DataProvider ;
                    sulo:hasFeature [ a sulo:InformationObject ;
                            sulo:hasValue "DP-001-HOSP" ] ] ],
        [ a :CareHandlingRole ;
            sulo:isFeatureOf [ a :CareProviderInstitute ;
                    sulo:hasFeature [ a sulo:InformationObject ;
                            sulo:hasValue :careHandling2025 ] ] ],
        [ a :PatientRole ;
            sulo:isFeatureOf [ a :Patient ;
                    sulo:hasFeature [ a sulo:In